In [ ]:
#%pip install -i https://pypi.gurobi.com gurobipy

In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import tuplelist
import itertools  
import random


In [2]:
import pandas as pd

object = pd.read_pickle(r'data/top25_dfs.pickle')

In [44]:
data = object[1]
data = data.transpose()
players = data.keys()
len(data)

26

In [45]:
l = len(players)

In [46]:
rankDic = {}

for i in range(0,25):
    row = data.iloc[i, :]
    for j in range(0,l):
        rankDic[(i,players[j])] = row[j]
    #rank_list.append(list(first_column))


In [47]:
itemList = data.keys()
itemList = itemList[0:l]
itemList

Index(['Adam Thielen', 'Mike Evans', 'JuJu Smith-Schuster', 'Albert Wilson',
       'Odell Beckham Jr.', 'Jarvis Landry', 'Courtland Sutton',
       'Mohamed Sanu', 'Emmanuel Sanders', 'Tyler Boyd', 'Amari Cooper',
       'Robert Woods', 'Michael Thomas', 'Marvin Jones', 'Anthony Miller',
       'John Brown', 'Tyreek Hill', 'Julio Jones', 'Tre'Quan Smith',
       'DeAndre Hopkins', 'Tyrell Williams', 'Dante Pettis', 'DeSean Jackson',
       'Cooper Kupp', 'Alshon Jeffery', 'Julian Edelman', 'Josh Gordon',
       'Donte Moncrief', 'Stefon Diggs', 'Brandin Cooks', 'Sammy Watkins',
       'Sterling Shepard', 'Christian Kirk', 'Calvin Ridley',
       'Larry Fitzgerald', 'Corey Davis', 'Curtis Samuel', 'Robby Anderson',
       'Allen Robinson', 'DaeSean Hamilton', 'Davante Adams', 'Dede Westbrook',
       'Marquez Valdes-Scantling', 'Will Fuller', 'T.Y. Hilton',
       'Geronimo Allison', 'Cole Beasley', 'Michael Gallup', 'Chris Godwin',
       'Tyler Lockett', 'Jamison Crowder', 'Keenan Al

In [48]:
Groups = [0,1]
numGrp = len(Groups)

G = []
CG = []
for i in range(0,numGrp):
    g  = []
    G.append(g)
    cg = []
    CG.append(cg)
    
row = data.iloc[25, :]
for i in range(0,len(row)):
    G[row[i]].append(players[i])
    

for i in range(0,numGrp):  
    for j in range(0,numGrp):
        if i != j :
            CG[i].extend(G[j]) 

In [49]:
p = []
for i in range(0,len(G)):
    pi = len(G[i])/len(itemList)
    p.append(pi)


In [50]:
p

[0.45454545454545453, 0.5454545454545454]

In [51]:

d = [0, 0, 0,0]

In [52]:
combinations = [p for p in itertools.product(itemList, repeat=2)]
len(combinations)

3025

In [53]:
scores = []
for tup in combinations:
    x,y = tup
    v = 0
    for i in range(0,25):
        if rankDic[(i,x)] < rankDic[(i,y)]:
            v = v +  1
    scores.append(v)

In [54]:

res = dict(zip(combinations, scores)) 

combinations, scores = gp.multidict(res)

In [55]:
# Declare and initialize model
m = gp.Model('RAP')

In [56]:
# Create decision variables for the RAP model
x = m.addVars(combinations,vtype=GRB.BINARY,name='x')

In [57]:
# Create  constraints
con = m.addConstrs((x[i,i] == 0  for i in itemList ), name='c0')

In [58]:
# Create  constraints
con = m.addConstrs((x[i,j] + x[j,i]  == 1   for j in itemList for i in itemList if i != j), name='c1')

In [59]:
# Create  constraints
con = m.addConstrs((x[i,j] + x[j,k] + x[k,i]  >= 1 for j in itemList for i in itemList for k in itemList if (i!= j and j!= k and k != i)), name='c2')

In [60]:
numGrp = 2
Groups = [0,1]

In [61]:
a = m.addVars(2,numGrp, name="a")

In [62]:
m.addConstrs( a[0,k] == (gp.quicksum(x[i, j] for i in G[k] for j   in  CG[k]) - gp.quicksum(x[j, i] for i in G[k] for j  in CG[k])) for k in Groups)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [63]:
m.addConstrs(a[1,k] == gp.abs_(a[0,k]) for k in Groups )

{0: <gurobi.GenConstr *Awaiting Model Update*>,
 1: <gurobi.GenConstr *Awaiting Model Update*>}

In [64]:
con = m.addConstrs( (a[1,k] <=  d[k]  for k in Groups ) ,name='cfair1')

In [65]:
# Objective: maximize total matching score of all assignments
m.setObjective(x.prod(scores), GRB.MINIMIZE)

In [66]:
# Save model for inspection
m.write('RAP.lp')

In [67]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 160439 rows, 3029 columns and 481229 nonzeros
Model fingerprint: 0x3ec3e3d9
Model has 2 general constraints
Variable types: 4 continuous, 3025 integer (3025 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 107968 rows and 1544 columns
Presolve time: 0.60s
Presolved: 52471 rows, 1485 columns, 158160 nonzeros
Variable types: 0 continuous, 1485 integer (1485 binary)
Found heuristic solution: objective 11225.000000

Root relaxation: objective 2.437000e+03, 21 iterations, 0.12 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2437.00000    0    2 11225.0000 2437.00000  78.3%     -    

In [68]:
# Display optimal values of decision variables
i = 0
s = 0
scoredict = {}
for v in m.getVars():
   
    i = i + 1
    s = s + v.x
    #print(v.varName, v.x)
    if i % len(itemList) == 0:
        #print(i / len(itemList))
        g = 0
        if (row[int(i / len(itemList)) - 1] == 1):
            g = 1
        scoredict[itemList[int(i / len(itemList)) - 1]] = (s,g)
        s = 0 

# Display optimal total matching score
#print('Total matching score: ', m.objVal)

In [69]:
scoredict

{'Adam Thielen': (9.0, 1),
 'Mike Evans': (4.0, 1),
 'JuJu Smith-Schuster': (3.0, 0),
 'Albert Wilson': (49.0, 0),
 'Odell Beckham Jr.': (0.0, 0),
 'Jarvis Landry': (29.0, 0),
 'Courtland Sutton': (38.0, 0),
 'Mohamed Sanu': (50.0, 0),
 'Emmanuel Sanders': (47.0, 1),
 'Tyler Boyd': (17.0, 0),
 'Amari Cooper': (14.0, 1),
 'Robert Woods': (18.0, 1),
 'Michael Thomas': (2.0, 1),
 'Marvin Jones': (28.0, 1),
 'Anthony Miller': (51.0, 1),
 'John Brown': (41.0, 0),
 'Tyreek Hill': (6.0, 0),
 'Julio Jones': (8.0, 1),
 "Tre'Quan Smith": (53.0, 1),
 'DeAndre Hopkins': (1.0, 1),
 'Tyrell Williams': (32.0, 0),
 'Dante Pettis': (35.0, 1),
 'DeSean Jackson': (40.0, 1),
 'Cooper Kupp': (20.0, 1),
 'Alshon Jeffery': (25.0, 1),
 'Julian Edelman': (11.0, 0),
 'Josh Gordon': (21.0, 0),
 'Donte Moncrief': (43.0, 0),
 'Stefon Diggs': (12.0, 0),
 'Brandin Cooks': (10.0, 0),
 'Sammy Watkins': (31.0, 0),
 'Sterling Shepard': (33.0, 1),
 'Christian Kirk': (39.0, 1),
 'Calvin Ridley': (30.0, 1),
 'Larry Fitzger

In [70]:
detOut = []
group = []
for j in range(0,len(itemList)):
    detOut.append(0)
    group.append(0)

i = 0

for item in scoredict.items():
    detOut[int(item[1][0])] = i
    group[int(item[1][0])] = int(item[1][1])
    i = i + 1

In [71]:
for w in sorted(scoredict, key=scoredict.get, reverse=False):
    print(w, scoredict[w])

Odell Beckham Jr. (0.0, 0)
DeAndre Hopkins (1.0, 1)
Michael Thomas (2.0, 1)
JuJu Smith-Schuster (3.0, 0)
Mike Evans (4.0, 1)
Davante Adams (5.0, 1)
Tyreek Hill (6.0, 0)
Keenan Allen (7.0, 0)
Julio Jones (8.0, 1)
Adam Thielen (9.0, 1)
Brandin Cooks (10.0, 0)
Julian Edelman (11.0, 0)
Stefon Diggs (12.0, 0)
Tyler Lockett (13.0, 1)
Amari Cooper (14.0, 1)
Kenny Golladay (15.0, 1)
Chris Godwin (16.0, 1)
Tyler Boyd (17.0, 0)
Robert Woods (18.0, 1)
T.Y. Hilton (19.0, 0)
Cooper Kupp (20.0, 1)
Josh Gordon (21.0, 0)
Mike Williams (22.0, 0)
Dede Westbrook (23.0, 0)
Allen Robinson (24.0, 1)
Alshon Jeffery (25.0, 1)
Will Fuller (26.0, 0)
D.J. Moore (27.0, 1)
Marvin Jones (28.0, 1)
Jarvis Landry (29.0, 0)
Calvin Ridley (30.0, 1)
Sammy Watkins (31.0, 0)
Tyrell Williams (32.0, 0)
Sterling Shepard (33.0, 1)
Corey Davis (34.0, 0)
Dante Pettis (35.0, 1)
Curtis Samuel (36.0, 1)
Robby Anderson (37.0, 1)
Courtland Sutton (38.0, 0)
Christian Kirk (39.0, 1)
DeSean Jackson (40.0, 1)
John Brown (41.0, 0)
Larry F